In [1]:
# !pip install datasets>=1.18.3
# !pip install transformers==4.11.3
# !pip install librosa
# !pip install jiwer
# !pip install -U accelerate --quiet
# !apt install git-lfs


In [2]:
yaml_train = "/kaggle/input/must-c-en-ar/train/txt/train.yaml"
transcript_train = "/kaggle/input/must-c-en-ar/train/txt/train_en.txt"
wav_dir_train = "/kaggle/input/must-c-en-ar/train/wav"

yaml_dev = "/kaggle/input/must-c-en-ar/dev/txt/dev.yaml"
transcript_dev = "/kaggle/input/must-c-en-ar/dev/txt/dev.en"
wav_dir_dev = "/kaggle/input/must-c-en-ar/dev/wav"

In [3]:
import yaml
from yaml import CLoader
import os
import re
def get_yaml_data(path):
    with open(path) as f:
        data = yaml.load(f, Loader=CLoader)
        return data    

def get_text_data(path):
    with open(path, 'rt', encoding='utf-8', errors='ignors') as f:
        return f.readlines()
    
def cleanify(s):
    s = re.sub(r"\([\sa-zA-Z]+\)+", " ", s)
    s = re.sub(r"[^\sa-zA-Z0-9?!'.]+", "", s)
    s = re.sub(r'[.]+', ".", s)
    s = re.sub(r'[" "]+', " ", s)

    s = s.rstrip().strip()

    return s

ImportError: cannot import name 'CLoader' from 'yaml' (C:\Users\20101\AppData\Roaming\Python\Python310\site-packages\yaml\__init__.py)

In [4]:
import pandas as pd

train_data = get_yaml_data(yaml_train)
txt = get_text_data(transcript_train)

train_data = [{"input_values" : (os.path.join(wav_dir_train, x["wav"]) , x["duration"] , x["offset"] ) , "labels" : cleanify(txt[i]) } for i , x in enumerate(train_data)]

dev_data = get_yaml_data(yaml_dev)
txt = get_text_data(transcript_dev)

dev_data = [{"input_values" : (os.path.join(wav_dir_dev, x["wav"]) , x["duration"] , x["offset"] ) , "labels" : cleanify(txt[i]) } for i , x in enumerate(dev_data)]


In [5]:
train_data = train_data[:1000]

In [7]:
vocab_list = list(set(" ".join([x["labels"] for x in train_data])))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict;

{'X': 0,
 '.': 1,
 '3': 2,
 'i': 3,
 "'": 4,
 'T': 5,
 'C': 6,
 'E': 7,
 'f': 8,
 'e': 9,
 'y': 10,
 '2': 11,
 'Z': 12,
 '6': 13,
 'I': 14,
 'x': 15,
 'H': 16,
 'J': 17,
 'l': 18,
 'n': 19,
 'A': 20,
 'b': 21,
 'F': 22,
 'D': 23,
 'w': 24,
 'O': 25,
 '!': 26,
 'B': 27,
 '5': 28,
 '9': 29,
 'S': 30,
 'G': 31,
 'z': 32,
 'g': 33,
 'm': 34,
 'L': 35,
 's': 36,
 'q': 37,
 '7': 38,
 'p': 39,
 'v': 40,
 ' ': 41,
 'o': 42,
 'W': 43,
 'j': 44,
 '?': 45,
 'Y': 46,
 'u': 47,
 'R': 48,
 'U': 49,
 'K': 50,
 'c': 51,
 'h': 52,
 '8': 53,
 'V': 54,
 '0': 55,
 'a': 56,
 'd': 57,
 'N': 58,
 '4': 59,
 'P': 60,
 't': 61,
 'M': 62,
 '1': 63,
 'r': 64,
 'k': 65}

In [8]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [9]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

68


In [10]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [11]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/kaggle/working/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [12]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [13]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [14]:
import torch
import torchaudio

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None
    
    def load_wav(self,wav_path,duration,offset):
        duration = int(float(duration) * 16000) 
        offset = int(float(offset) * 16000)
        wav, _ = torchaudio.load(wav_path,frame_offset=offset, num_frames=duration)
        return wav[0]
    
    def conver_label(self,text):
        with self.processor.as_target_processor():
            en = self.processor(text).input_ids
        return en
    
    def __call__(self, features):
        
        input_features = [{"input_values": self.load_wav(*feature["input_values"])} for feature in features]
        label_features = [{"input_ids": self.conver_label(feature["labels"])} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [15]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [17]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [18]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

In [ ]:
model.freeze_feature_extractor()

In [22]:
model.lm_head = torch.nn.Linear(768,len(vocab_dict))

In [23]:
model.config.vocab_size = len(vocab_dict)

In [24]:
model.config.vocab_size

68

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/kaggle/working/output/",
  group_by_length=True,
  per_device_train_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=False,
  gradient_checkpointing=True,
#   gradient_accumulation_steps = 10,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)


In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=processor.feature_extractor,
)


In [ ]:
trainer.train()